In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain_community.vectorstores import FAISS

In [ ]:
DATA_PATH = "data"
DB_FAISS_PATH = "vectorstores/faiss_db"

# Create a Vector DB
def create_vector_db(data):
    loader = DirectoryLoader(DATA_PATH, glob="*.pdf", loader_cls=PyPDFLoader)
    documents =  loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
    chunks = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2",
    model_kwargs = {
        'device': 'cpu',
    })
    db = FAISS.from_documents(chunks, embeddings)
    db.save_local(DB_FAISS_PATH)
    return db

In [ ]:
db = create_vector_db(DATA_PATH)

In [ ]:
custom_prompt = """
Use the following prompt to retrieve information from the database:
Question: {question}
Context: {context}

Answer:
"""

def set_custom_prompt():
    prompt = PromptTemplate(template=custom_prompt, input_variables=["question", "context"])
    return prompt

def load_llm():
    llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q8_0.bin", 
                        model_type="llama",
                        max_new_tokens=512,
                        tempterature=0.5,)
    return llm

def retrieve_answer(llm, prompt, db):
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                           chain_type="stuff",
                                           retriever = db.as_retriever(search_kwargs={'k': 2}),
                                           return_source_documents=True,
                                           chain_type_kwargs={'prompt': prompt}
                                           )
    return qa_chain

def qa_bot():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2",
                                        model_kwargs = {
                                             'device': 'cpu',
                                        })
    db = FAISS.load_local(DB_FAISS_PATH, embeddings, allow_dangerous_deserialization=True)
    llm = load_llm()
    qa_prompt = set_custom_prompt()
    qa = retrieve_answer(llm, qa_prompt, db)
    return qa

def final_result(query):
    result = qa_bot()
    response = result({'question': query})
    return response



In [ ]:
# set custom prompt
prompt = set_custom_prompt()

In [ ]:
llm =load_llm()

In [ ]:
retrieve_answer(llm=llm, prompt=prompt, db=db)


In [74]:
def retrieve_answer(llm, prompt, db):
    qa = RetrievalQA()
    return qa

retrieve_answer(llm=llm, prompt=prompt, db=db)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/zaineelmithani/anaconda3/envs/medicalChatbot/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/jk/x7843gt92gn5x1xqygl255_w0000gn/T/ipykernel_88519/2114493568.py", line 5, in <module>
    retrieve_answer(llm=llm, prompt=prompt, db=db)
  File "/var/folders/jk/x7843gt92gn5x1xqygl255_w0000gn/T/ipykernel_88519/2114493568.py", line 2, in retrieve_answer
    qa = RetrievalQA(retriever=db.as_retriever(), combine_documents_chain={'llm': llm, 'prompt': prompt}, combine_documents_chain_kwargs={'max_new_tokens': 512, 'temperature': 0.5, 'top_p': 0.95, 'max_tokens': 512, 'num_return_sequences': 1, 'stop_sequences': ['\n']}, combine_documents_chain_type='llm', combine_documents_chain_type_kwargs={'model': 'gpt2', 'max_new_tokens': 512, 'temperature': 0.5, 'top_p': 0.95, 'max_tokens': 512, 'num_return_sequences': 1, 'stop_sequences': ['\n']},